In [1]:
import os
import sys
import re
from tqdm import tqdm
import numpy as np
from scipy.io import loadmat
from PIL import Image

from pycocotools.coco import COCO
coco = COCO('D://datasets/coco/annotations/instances_val2017.json')


loading annotations into memory...
Done (t=0.73s)
creating index...
index created!


In [2]:
dataset_dir = 'D://datasets/wider'
ann_dir = os.path.join(dataset_dir, 'annotations')


In [3]:
# cats
# coco.cats[coco.getCatIds()[0]]
# {'id': 1, 'name': 'person', 'supercategory': 'person'}

# imgs
# coco.imgs[coco.getImgIds()[0]]
# {'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
#  'date_captured': '2013-11-14 17:02:52',
#  'file_name': '000000397133.jpg',
#  'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
#  'height': 427,
#  'id': 397133,
#  'license': 4,
#  'width': 640}

# anns
# coco.anns[coco.getAnnIds()[0]]
# {'area': 702.1057499999998,
#  'bbox': [473.07, 395.93, 38.65, 28.67],
#  'category_id': 18,
#  'id': 1768,
#  'image_id': 289343,
#  'iscrowd': 0,
#  'segmentation': <TRUNCATED>}

# catToImgs
# coco.catToImgs[coco.getCatIds()[0]]
# [list of img_id]

# imgToAnns
# coco.imgToAnns[coco.getCatIds()[0]]
# [list of anns]


In [4]:
wider_dir = os.listdir(ann_dir)
wider_dir

['readme.txt',
 'wider_face_test.mat',
 'wider_face_test_filelist.txt',
 'wider_face_train.mat',
 'wider_face_train_bbx_gt.txt',
 'wider_face_val.mat',
 'wider_face_val_bbx_gt.txt']

In [5]:
train_data = loadmat(os.path.join(ann_dir, wider_dir[5]))

In [6]:
with open(os.path.join(ann_dir, wider_dir[0]), 'r') as f:
    print(f.read())

train_data.keys()

Attached the mappings between attribute names and label values.

blur:
  clear->0
  normal blur->1
  heavy blur->2

expression:
  typical expression->0
  exaggerate expression->1

illumination:
  normal illumination->0
  extreme illumination->1

occlusion:
  no occlusion->0
  partial occlusion->1
  heavy occlusion->2

pose:
  typical pose->0
  atypical pose->1

invalid:
  false->0(valid image)
  true->1(invalid image)

The format of txt ground truth.
File name
Number of bounding box
x1, y1, w, h, blur, expression, illumination, invalid, occlusion, pose


dict_keys(['__header__', '__version__', '__globals__', 'blur_label_list', 'event_list', 'expression_label_list', 'face_bbx_list', 'file_list', 'illumination_label_list', 'invalid_label_list', 'occlusion_label_list', 'pose_label_list'])

In [7]:
"""
- event_list
type of event
also name of folder in images

- file_list
list of files
to get this file, {wider_root}/images/{event_type}/{file_name}

- 
"""

'\n- event_list\ntype of event\nalso name of folder in images\n\n- file_list\nlist of files\nto get this file, {wider_root}/images/{event_type}/{file_name}\n\n- \n'

In [8]:
train_data['file_list'][0, 0][0, 0].shape, \
train_data['face_bbx_list'][0, 0][0, 0].shape, \
train_data['blur_label_list'][0, 0][0, 0].shape, \
train_data['expression_label_list'][0, 0][0, 0].shape, \
train_data['illumination_label_list'][0, 0][0, 0].shape, \
train_data['occlusion_label_list'][0, 0][0, 0].shape, \
train_data['pose_label_list'][0, 0][0, 0].shape, \
train_data['invalid_label_list'][0, 0][0, 0].shape

((1,), (126, 4), (126, 1), (126, 1), (126, 1), (126, 1), (126, 1), (126, 1))

In [9]:
def parse_int_from_string(s):
    all_digits = ''.join(filter(str.isdigit, s))
    if all_digits == '':
        return None
    else:
        return int(all_digits)


In [10]:
def get_labels():
    return {
        'blur': {
            '0': {'id': 0, 'name': 'clear' , 'tag': 'blur'},
            '1': {'id': 1, 'name': 'normal', 'tag': 'blur'},
            '2': {'id': 2, 'name': 'heavy' , 'tag': 'blur'}
        },
        'expression': {
            '0': {'id': 0, 'name': 'typical'   , 'tag': 'expression'},
            '1': {'id': 1, 'name': 'exaggerate', 'tag': 'expression'}
        },
        'illumination': {
            '0': {'id': 0, 'name': 'normal' , 'tag': 'expression'},
            '1': {'id': 1, 'name': 'extreme', 'tag': 'expression'}
        },
        'occlusion': {
            '0': {'id': 0, 'name': 'none' , 'tag': 'illumination'},
            '1': {'id': 1, 'name': 'partial', 'tag': 'illumination'},
            '2': {'id': 2, 'name': 'heavy' , 'tag': 'illumination'}
        },
        'pose': {
            '0': {'id': 0, 'name': 'typical' , 'tag': 'pose'},
            '1': {'id': 1, 'name': 'atypical', 'tag': 'pose'}
        },
        'invalid': {
            '0': {'id': 0, 'name': 'false', 'tag': 'invalid'},
            '1': {'id': 1, 'name': 'true' , 'tag': 'invalid'}
        },
    }

def get_tags():
    return list(get_labels().keys())

def get_events(data):
    event_list = [str(event[0]) for event in data['event_list'][:, 0]]
    return event_list

labels = get_labels()
tags   = get_tags()
events = get_events(train_data)


In [13]:
def get_num_imgs(data):
    img_count = 0
    
    for event_id, event in enumerate(get_events(data)):
        file_list = [str(f[0][0]) for f in train_data['file_list'][event_id, 0]]
        img_count += len(file_list)
    
    return img_count
    

def get_img_infos(data):
    all_img_info = {}
    img_count = 0
    
    with tqdm(total=get_num_imgs(data)) as pbar:
    
        for event_id, event in enumerate(get_events(data)):
            file_list = [str(f[0][0]) for f in train_data['file_list'][event_id, 0]]

            for file_id, file_name in enumerate(file_list):
                # Get image info
                img_info = {
                    'id'       : img_count,
                    'event'    : event,
                    'file_path': os.path.join('images', event, file_name) + '.jpg'
                }

                img = Image.open(os.path.join(dataset_dir, img_info['file_path']))
                img = np.array(img)

                img_info['height'] = img.shape[1]
                img_info['width']  = img.shape[0]

                # Store info
                all_img_info[img_count] = img_info

                # Update progress
                img_count += 1
                pbar.update(1)
    
    return all_img_info

# imgs
# coco.imgs[coco.getImgIds()[0]]
# {'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg',
#  'date_captured': '2013-11-14 17:02:52',
#  'file_name': '000000397133.jpg',
#  'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg',
#  'height': 427,
#  'id': 397133,
#  'license': 4,
#  'width': 640}

img_infos = get_img_infos(train_data)
# get_num_imgs(train_data)


100%|██████████████████████████████████████████████████████████████████████████████| 3226/3226 [01:20<00:00, 40.11it/s]


In [35]:
def get_anns(data):
    all_img_to_anns = {}
    all_anns = {}
    img_count = 0
    ann_count = 0
    
    for event_id, event in enumerate(get_events(data)):
            file_list = [str(f[0][0]) for f in train_data['file_list'][event_id, 0]]

            for file_id, file_name in enumerate(file_list):
                anns_for_img = []
                
                for ann_idx, ann in enumerate(data['face_bbx_list'][event_id, 0][file_id, 0]):
                    # Get ann info
                    ann_info = {
                        'id'      : ann_count,
                        'image_id': img_count,
                        'area'    : ann[2] * ann[3],
                        'bbox'    : ann.tolist(),
                    }
                    
                    for tag in get_tags():
                        ann_info[tag] = data[tag + '_label_list'][event_id, 0][file_id, 0][ann_idx, 0]
                    
                    # Store info
                    anns_for_img.append(ann_info)
                    all_anns[ann_count] = ann_info
                    
                    # Update progress
                    ann_count += 1
                
                # Store info
                all_img_to_anns[img_count] = anns_for_img
                
                # Update progress
                img_count += 1
    
    return all_img_to_anns, all_anns
    

# anns
# coco.anns[coco.getAnnIds()[0]]
# {'area': 702.1057499999998,
#  'bbox': [473.07, 395.93, 38.65, 28.67],
#  'category_id': 18,
#  'id': 1768,
#  'image_id': 289343,
#  'iscrowd': 0,
#  'segmentation': <TRUNCATED>}

# imgToAnns
# coco.imgToAnns[coco.getCatIds()[0]]
# [list of anns]

imgToAnns , anns = get_anns(train_data)


In [22]:
# img_infos.keys()

In [25]:
# coco.cats

In [37]:
imgToAnns[0]

[{'area': 16,
  'bbox': [345, 211, 4, 4],
  'blur': 2,
  'expression': 0,
  'id': 0,
  'illumination': 0,
  'image_id': 0,
  'invalid': 0,
  'occlusion': 2,
  'pose': 0},
 {'area': 9,
  'bbox': [331, 126, 3, 3],
  'blur': 0,
  'expression': 0,
  'id': 1,
  'illumination': 0,
  'image_id': 0,
  'invalid': 1,
  'occlusion': 0,
  'pose': 0},
 {'area': 12,
  'bbox': [250, 126, 3, 4],
  'blur': 2,
  'expression': 0,
  'id': 2,
  'illumination': 0,
  'image_id': 0,
  'invalid': 0,
  'occlusion': 2,
  'pose': 0},
 {'area': 20,
  'bbox': [221, 128, 4, 5],
  'blur': 0,
  'expression': 0,
  'id': 3,
  'illumination': 0,
  'image_id': 0,
  'invalid': 1,
  'occlusion': 0,
  'pose': 0},
 {'area': 12,
  'bbox': [427, 116, 3, 4],
  'blur': 2,
  'expression': 0,
  'id': 4,
  'illumination': 0,
  'image_id': 0,
  'invalid': 0,
  'occlusion': 2,
  'pose': 0},
 {'area': 12,
  'bbox': [393, 79, 3, 4],
  'blur': 0,
  'expression': 0,
  'id': 5,
  'illumination': 0,
  'image_id': 0,
  'invalid': 1,
  'occlu